# INSTALLING LIBRARIES

In [1]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# LOADING THE DATASETS

In [2]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)), # RESIZE THE IMAGES 
                                     transforms.ToTensor(), # CONVERTING THEM TO TENSORS
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010]) # IMAGE NORMALIZATION
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing - ALLOWS US TO LOAD DATA IN BATCHES so as not to overwhelm our computing powers
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

100%|██████████| 170498071/170498071 [00:01<00:00, 94870613.85it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# CNN ARCHITECTURE

For **nn.Conv2d** - we define the channels they receive (3 if they are RGB images), the channels they will return, and the kernel size

for **nn.MaxPool2d** - max-pooling layer, it just requires the kernel size & stride

for **nn.linear** - fully connected layer & **nn.ReLU** is an activation function used

In [3]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

# SETTING HYPERPARAMETERS

In [4]:
model = ConvNeuralNet(num_classes).to(device)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

# TRAINING THE MODEL

In [5]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
# Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/20], Loss: 1.7663
Epoch [2/20], Loss: 1.3774
Epoch [3/20], Loss: 0.8064
Epoch [4/20], Loss: 1.0771
Epoch [5/20], Loss: 1.2523
Epoch [6/20], Loss: 0.9002
Epoch [7/20], Loss: 1.0478
Epoch [8/20], Loss: 1.2560
Epoch [9/20], Loss: 0.8017
Epoch [10/20], Loss: 1.4000
Epoch [11/20], Loss: 1.2817
Epoch [12/20], Loss: 0.7695
Epoch [13/20], Loss: 0.6671
Epoch [14/20], Loss: 1.0705
Epoch [15/20], Loss: 0.5806
Epoch [16/20], Loss: 0.6427
Epoch [17/20], Loss: 0.5489
Epoch [18/20], Loss: 0.8282
Epoch [19/20], Loss: 0.5758
Epoch [20/20], Loss: 0.3980


# TESTING THE MODEL

In [6]:
with torch.no_grad(): # THERE IS NO NEED TO CALCULATE GRADIENTS
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))

Accuracy of the network on the 50000 train images: 83.106 %
